In [4]:
from projectaria_tools.core.stream_id import StreamId
from projectaria_tools.core import calibration
from projectaria_tools.projects.adt import (
    AriaDigitalTwinDataProvider,
    AriaDigitalTwinSkeletonProvider,
    AriaDigitalTwinDataPathsProvider,
    bbox3d_to_line_coordinates,
    bbox2d_to_image_coordinates,
    utils as adt_utils,
)
import numpy as np
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
import torch
import matplotlib.pyplot as plt
import matplotlib.patches as patches

In [5]:
path = "../dataset/raw_dataset/Apartment_release_clean_seq131"

In [6]:
path_provider = AriaDigitalTwinDataPathsProvider(path)
data_paths = path_provider.get_datapaths_by_device_num(0)
dt_provider = AriaDigitalTwinDataProvider(data_paths)
stream = StreamId('214-1')

Loaded #closed loop trajectory poses records: 2880


[AriaDigitalTwinDataPathsProvider][WARNING]: Sequence has version 1.1, but dataset versions 1.X are deprecated. We recommend you update your data by redownloading.
[AriaDigitalTwinDataPathsProvider][WARNING]: no segmentation image data found at ../dataset/raw_dataset/Apartment_release_clean_seq131/1WM103600M1292_optitrack_release_clean_seq131/segmentations.vrs
[MultiRecordFileReader][DEBUG]: Opened file '../dataset/raw_dataset/Apartment_release_clean_seq131/1WM103600M1292_optitrack_release_clean_seq131/video.vrs' and assigned to reader #0
[VrsDataProvider][INFO]: streamId 211-1/camera-et activated
[VrsDataProvider][INFO]: streamId 214-1/camera-rgb activated
[VrsDataProvider][INFO]: streamId 247-1/baro0 activated
[VrsDataProvider][WARNING]: Unsupported TimeSync mode: APP, ignoring.
[VrsDataProvider][INFO]: Timecode stream found: 285-2
[VrsDataProvider][INFO]: streamId 1201-1/camera-slam-left activated
[VrsDataProvider][INFO]: streamId 1201-2/camera-slam-right activated
[VrsDataProvider]

Loaded #EyeGazes: 3526


er][DEBUG]: Opened file '../dataset/raw_dataset/Apartment_release_clean_seq131/1WM103600M1292_optitrack_release_clean_seq131/synthetic_video.vrs' and assigned to reader #0
[VrsDataProvider][INFO]: streamId 214-1/camera-rgb activated
[VrsDataProvider][INFO]: streamId 1201-1/camera-slam-left activated
[VrsDataProvider][INFO]: streamId 1201-2/camera-slam-right activated
[VrsDataProvider][INFO]: streamId 1202-1/imu-right activated
[DeviceCadExtrinsics][WARNING]: No CAD available for simulated device
[RecordReaderInterface][ERROR]: Tag 'metadata' was not found in the VRS file tags
[AriaDigitalTwinDataProvider][INFO]: skip loading skeletonMetaDataFilePath because the data path is empty
[AriaDigitalTwinDataProvider][INFO]: skip loading skeletonsFilePaths because the data path is empty
Failed to parse eye gaze vergence file: Extra column "yaw_rads_cpf" in header of file "../dataset/raw_dataset/Apartment_release_clean_seq131/1WM103600M1292_optitrack_release_clean_seq131/eyegaze.csv".


In [7]:
LENGTH_OF_SEQUENCE = 300
STRIDE = 3
SKIP_N_PIXELS = 10

In [8]:
timestamps = dt_provider.get_aria_device_capture_timestamps_ns(stream)
l = len(timestamps) // 2
r = l + LENGTH_OF_SEQUENCE
timestamps = timestamps[l:r:STRIDE]
cam_calib = dt_provider.get_aria_camera_calibration(stream)

In [9]:
def get_point_cloud(timestamp):
    counter = 0
    aria_3d_pose = dt_provider.get_aria_3d_pose_by_timestamp_ns(timestamp).data()
    scene_cam_matrix = aria_3d_pose.transform_scene_device @ cam_calib.get_transform_device_camera()

    depth_image = dt_provider.get_depth_image_by_timestamp_ns(timestamp, stream).data().to_numpy_array()
    RGB_image = dt_provider.get_aria_image_by_timestamp_ns(timestamp, stream).data().to_numpy_array()
    synthetic_RGB_image = dt_provider.get_synthetic_image_by_timestamp_ns(timestamp, stream).data().to_numpy_array()

    img_points_in_scene = []
    img_cols = []
    synthetic_cols = []

    for u in range(RGB_image.shape[1]):
        for v in range(RGB_image.shape[0]):
            counter += 1
            if counter % SKIP_N_PIXELS != 0:
                continue

            ray = cam_calib.unproject([u, v])
            if ray is None:
                continue

            d = depth_image[v, u] / 1000
            p_in_cam = d * ray
            p_in_scene = scene_cam_matrix @ p_in_cam

            img_points_in_scene.append(p_in_scene)
            img_cols.append(RGB_image[v, u] / 256)
            synthetic_cols.append(synthetic_RGB_image[v, u] / 256)

    return img_points_in_scene, img_cols, synthetic_cols

In [10]:
points = []
colors = []
synthetic_colors = []
for i in tqdm(range(len(timestamps))):
    tmp_p, tmp_c, tmp_s_c = get_point_cloud(timestamps[i])
    points.append(tmp_p)
    colors.append(tmp_c)
    synthetic_colors.append(tmp_s_c)

points = np.array(points)


100%|████████████████████████████████████████████████████████████████████| 100/100 [07:30<00:00,  4.50s/it]


In [11]:
points[:, :, [0, 1, 2]] = points[:, :, [2, 0, 1]]

In [ ]:
import rerun as rr 
from rerun.datatypes import Angle, RotationAxisAngle
from math import pi
rr.init("point_cloud_sequence_visualization")
# rr.serve()

rr.set_time_seconds("stable_time", 0)
arrows = rr.Arrows3D(origins=[[0, 0, 0], [0, 0, 0], [0, 0, 0]], vectors=[[1, 0, 0], [0, 1, 0], [0, 0, 1]], labels=["X", "Y", "Z"])
rr.log("real/arrow", arrows)
rr.log("synthetic/arrow", arrows)

In [14]:
for i in tqdm(range(len(timestamps))):
    time = (timestamps[i] - timestamps[0]) / 1e9
    rr.set_time_seconds("stable_time", time)

    rr.log(
        "3D/real_point_cloud",
        rr.Points3D(points[i], colors=colors[i])
    )

    rr.log(
        "3D/synthetic_point_cloud",
        rr.Points3D(points[i], colors=synthetic_colors[i])
    )

    RGB_image = dt_provider.get_aria_image_by_timestamp_ns(timestamps[i], stream).data().to_numpy_array()
    RGB_image = RGB_image.transpose([1, 0, 2])
    rr.log(
        "2D/RGB_image",
        rr.Image(data=RGB_image)
    )

    synthetic_image = dt_provider.get_synthetic_image_by_timestamp_ns(timestamps[i], stream).data().to_numpy_array()
    synthetic_image = synthetic_image.tranpose([1, 0, 2])
    rr.log(
        "2D/

[2024-08-02T10:33:11Z INFO  re_sdk::web_viewer] Sleeping a short while to give the browser time to connect…
[2024-08-02T10:33:12Z INFO  re_sdk::web_viewer] Shutting down without any clients ever having connected. Consider sleeping to give them more time to connect
[2024-08-02T10:33:12Z INFO  re_ws_comms::server] Shutting down Rerun server on ws://localhost:9877 after serving 0 client(s)
[2024-08-02T10:34:52Z INFO  re_ws_comms::server] Hosting a WebSocket server on ws://localhost:9877. You can connect to this with a native viewer (`rerun ws://localhost:9877`) or the web viewer (with `?url=ws://localhost:9877`).
[2024-08-02T10:34:52Z INFO  re_sdk::web_viewer] Hosting a web-viewer at http://localhost:9090?url=ws://localhost:9877
100%|████████████████████████████████████████████████████████████████████| 100/100 [00:23<00:00,  4.33it/s]


In [ ]:
rr.save()